# INFO 2950 Final Project

In [ ]:
import pandas as pd
import seaborn

In [1]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

reserach question:

In [ ]:
pd.read_csv('./Data_2019/Incidents_Offenses_Victims_and_Known_Offenders_by_Bias_Motivation_2019.csv')